### <center><font color=navy> Tutorial #12 Computer- and robot-assisted surgery</font></center>
## <center><font color=navy> ML Basics IV/Generative Adversarial Networks (GANs)</font></center>
<center>&copy; Sebastian Bodenstedt & Micha Pfeiffer, National Center for Tumor Diseases (NCT) Dresden<br>
    <a href="https://www.nct-dresden.de/"><img src="https://www.nct-dresden.de/++theme++nct/images/logo-nct-en.svg"></a> </center>

## Introduction

Welcome to the lab focusing on *creating realistic synthetic surgical images with a sim2real Generative Adversarial Network (GAN)*. In this lab, we will generate images from synthetic surgical scenes using 3D models. We will then utilize a Generative Adversarial Network (GAN) to make these images look realistic. Furthermore, we will experiment with different styles and their effect on the output. The model used in this lab is based on the paper [*Generating large labeled data sets for
laparoscopic image processing tasks using
unpaired image-to-image translation*](https://arxiv.org/pdf/1907.02882.pdf) by Micha Pfeiffer et al.
![Network](http://sds.bodenstedt.eu/Network.png)

## Download and extract models and style images

To get started, we check if we already have downloaded the data archive, if not we download it a new from the internet and extract it.

In [ ]:
import urllib.request
from os.path import exists
import zipfile
import progressbar

pbar = None

def show_progress(block_num, block_size, total_size):
    global pbar
    if pbar is None:
        pbar = progressbar.ProgressBar(maxval=total_size)
        pbar.start()

    downloaded = block_num * block_size
    if downloaded < total_size:
        pbar.update(downloaded)
    else:
        pbar.finish()
        pbar = None

url = "http://sds.bodenstedt.eu/data.zip"
file_path = "data.zip"

if not exists(file_path): # does zip file already exist?
    urllib.request.urlretrieve(url, file_path, show_progress) # if not, download it
    with zipfile.ZipFile(file_path, 'r') as zip_ref: # and unzip it
        zip_ref.extractall(".")

## Load data into Python
The aim of this task is load the relevant data and models into Python and, via pre-processing, make the data ready for style transfer.

**Set up imports**

First we import all relevant dependencies into Python

In [ ]:
from torchvision import transforms
import cv2
import sys
import torch
from image2image.networks import AdaINGen, StylelessGen
import numpy as np
import image2image.helper as helper
from matplotlib import pyplot as plt

**Loading models into Python**

Then we load the file containing the parameters of the pre-trained encoders and decoders into memory.

In [ ]:
models = torch.load("Model/Image2Image.pt") # Load pre-trained models from file

Next, we instantiate the styleless encoder/decoder combination, feed it the pre-trained parameters and copy it to copy.

In [ ]:
model_a = StylelessGen(3, helper.param_model) # Instantiate Styleless encoder/decoder
model_a.load_state_dict(models['a']) # Load pre-trained parameters
model_a.cuda() # Move model to GPU
model_a = model_a.eval() # Set to evaluation mode

We then perform the same steps with the AdaIN encoder/decoder combination

In [ ]:
model_b = AdaINGen(3, helper.param_model) # Instantiate AdaIN encoder/decoder
model_b.load_state_dict(models['b']) # Load pre-trained parameters
model_b.cuda() # Move model to GPU
mobel_b = model_b.eval() # Set to evaluation mode

**Loading images into Pytorch**

Next, we use OpenCV to load an image of the synthetic that we want to apply a style to. Furthermore, we load an image that contains the style that we want to apply. Here you can experiment with loading your own images (or different ones from the provided data, it contains 652 images) and/or with loading different style images from different patients. The dataset contains 5 images from four patients (P1-P4).

In [ ]:
image = cv2.imread("images/img010.png") # Load synthetic image from file
style_image = cv2.imread("styles/P1/style0.png") # Load style image from file

# Display images
plt.xticks([]), plt.yticks([])  # Hides the graph ticks and x / y axis
plt.imshow(image[:,:,::-1])
plt.title("Synthetic Image")
plt.show()

plt.xticks([]), plt.yticks([])  # Hides the graph ticks and x / y axis
plt.imshow(style_image[:,:,::-1])
plt.title("Style Image")
plt.show()

**Image pre-processing**

In order to feed the images into the Pytorch networks, they need to be pre-processed first. For this, we write a function that first resizes the input image onto a size that is manageable for the GPU, then we convert the image from BGR-color notation (OpenCV Standard) to RGB-color notation. The image is then converted into a Pytorch tensor. The resulting tensor is then normalized to the range -1 to 1.

In [ ]:
def preprocess_image(image_in, out_size):
    image = cv2.resize(image_in, out_size, interpolation=cv2.INTER_LINEAR) # Resize image to wanted size
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert from BGR to RGB

    img = transforms.functional.to_tensor(image) # Convert to Pytorch tensor
    img = transforms.functional.normalize(img, (0.5,0.5,0.5), (0.5,0.5,0.5) ) # Normalize tensor to range -1 .. 1
    img = img.unsqueeze(0) # add batch dimension
    
    return img

The two images are then pre-processed. As image size, we used a resolution of 580x270 pixels. Afterwards, we transfer the images to the GPU.

In [ ]:
input_size = (580, 270) # Network input resolution

# Pre-process images
image_tensor = preprocess_image(image, input_size)
style_image_tensor = preprocess_image(style_image, input_size)

# Move image data to GPU
image_tensor = image_tensor.cuda()
style_image_tensor = style_image_tensor.cuda()

## Apply style transfer to image
Now that we have loaded the required neural networks as well as loaded and pre-processed the images, we can now continue with performing the style transfer. 

**Compact representation of the synthetic image**

First, using the encoder of the pre-trained styleless network, we calculate the compact representation of the synthetic image. ![Encoder](http://sds.bodenstedt.eu/t51.png)

In [ ]:
with torch.no_grad(): # Since we are not training the network, we don't need gradient information to be retained
    image_representation = model_a.encode(image_tensor) # Calculate compact representation for synthetic image

**Extract/generate style**

Using the pre-trained AdaIN network, we calculate the compact representation of the style image as well as the compact representation of the style. As we are only interested in the latter, we discard the former. ![Style encoder](http://sds.bodenstedt.eu/t52_1.png)![Random style](http://sds.bodenstedt.eu/t52_2.png)

In [ ]:
with torch.no_grad():
    _, style = model_b.encode(style_image_tensor) # Extract style from style image
    print("Extracted style vector:")
    for i in range(helper.param_model["style_dim"]):
        print(style[0, i, 0, 0].item())

Alternatively, we can also generate a random vector, representing a random style. For this, we draw from a standard normal distribution (mean 0, variance 1).

In [ ]:
style_random = torch.randn(1, helper.param_model["style_dim"], 1, 1) # Generate random style vector
print("Random style vector:")
for i in range(helper.param_model["style_dim"]):
    # style vector can be manually adjusted by assigning values to the different entries
    #style_random[0, i, 0, 0] = 0
    print(style_random[0, i, 0, 0].item())
style_random = style_random.cuda() # Move vector to GPU

**Apply style to synthetic image**

Next, we aim to apply a style (either the extracted style or a random style) to the compact representation of the synthetic image. For this, we feed both the compact representation and a style vector into the decoder of the AdaIN network.
![Decoder](http://sds.bodenstedt.eu/t53.png)

In [ ]:
with torch.no_grad():
    stylized_image = model_b.decode(image_representation, style) # Apply extracted style to compact representation of synthetic image
    
    stylized_image_random = model_b.decode(image_representation, style_random) # Apply randomly generated style to compact representation of synthetic image

**Visualize results**

To visualize the resulting images, we first need to convert them back into a standard image format (e.g. OpenCV/NumPy). For this, we write a function that first removes the extraneous batch dimension. We then reverse the normalization, converting the pixel values to a range of 0 to 1. The result is then copied back into RAM and converted into a NumPy data structure. We then rearrange the dimension into $height \times width \times channels$ and scale the pixel values to a range of 0 to 255. The image is then converted into a 8-bit data format and we convert it from RGB to BGR.

In [ ]:
def convert_image(image_in):
    image = image_in.squeeze(0) # Remove batch dimension
    image = transforms.functional.normalize(image, (-1,-1,-1), (2,2,2) ) # adjust to range 0 .. 1
    image = image.cpu().numpy() # Copy data to CPU and convert to NumPy

    image = np.transpose(image, (1, 2, 0))*255 # Adjust order of the dimensions and scale to 0 .. 255
    image = image.astype(np.uint8) # Convert to 8 bit data type
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # convert RGB to BGR

    return image

We apply the function to the generated images and visualize the results.

In [ ]:
# Convert stylized images to OpenCV images
stylized_image_cv = convert_image(stylized_image) 
stylized_image_random_cv = convert_image(stylized_image_random)

# Visualize results
plt.xticks([]), plt.yticks([])  # Hides the graph ticks and x / y axis
plt.imshow(image[:,:,::-1])
plt.title("Synthetic Image")
plt.show()

plt.xticks([]), plt.yticks([])  # Hides the graph ticks and x / y axis
plt.imshow(style_image[:,:,::-1])
plt.title("Style Image")
plt.show()

plt.xticks([]), plt.yticks([])  # Hides the graph ticks and x / y axis
plt.imshow(stylized_image_cv[:,:,::-1])
plt.title("Synthetic Image with extracted style")
plt.show()

plt.xticks([]), plt.yticks([])  # Hides the graph ticks and x / y axis
plt.imshow(stylized_image_random_cv[:,:,::-1])
plt.title("Synthetic Image with random style")
plt.show()

## Experiment with different styles
Now that we have setup an entire pipeline for extracting styles/generating random styles and applying those styles to synthetic images, we can experiment with extracting different styles. For this, try modifying which images are loaded for styles extraction. Also consider adding your own images (don't have to be surgery related). Alternative you can also manually modify the style. 